In [1]:
import sys
sys.path.insert(0, "/home/aisummer/ssemenov_workspace/nlp-service")

import pandas as pd 
import ast
import numpy as np
import json
import os
import nltk

from src.DocumentsParser.utils import DBS_DIR_DENSE_VECTORDB_NAME, DBS_DIR_SPARSE_VECTORDB_NAME
from src.evaluation_metrics import ReaderMetrics
from src.DocumentsSummarizer.Summarizer import SummarizerModule
from src.LLM_build.utils import llmConfig
from src.utils import DialogueState
from src.logger import Logger

### 1) Создаём объекты

In [2]:
reader = SummarizerModule()
config = DialogueState()
configLLM = llmConfig()

options = {
"num_keep": configLLM.num_keep,
"seed": configLLM.seed,
"num_predict": configLLM.num_predict,
"top_k": configLLM.top_k,
"top_p": configLLM.top_p,
"tfs_z": configLLM.tfs_z,
"typical_p": configLLM.typical_p,
"repeat_last_n": configLLM.repeat_last_n,
"temperature": configLLM.temperature,
"repeat_penalty": configLLM.repeat_penalty,
"presence_penalty": configLLM.presence_penalty,
"frequency_penalty": configLLM.frequency_penalty,
"mirostat": configLLM.mirostat,
"mirostat_tau": configLLM.mirostat_tau,
"mirostat_eta": configLLM.mirostat_eta,
"penalize_newline": configLLM.penalize_newline,
#"stop": config.stop,
"numa": configLLM.numa,
"num_ctx": configLLM.num_ctx,
"num_batch": configLLM.num_batch,
"num_gpu": configLLM.num_gpu,
"main_gpu": configLLM.main_gpu,
"low_vram": configLLM.low_vram,
"f16_kv": configLLM.f16_kv,
"vocab_only": configLLM.vocab_only,
"use_mmap": configLLM.use_mmap,
"use_mlock": configLLM.use_mlock,
"num_thread": configLLM.num_thread
}

### 2) Указываем инфо для данных

In [3]:
BENCHMARKS_INFO = {'sberquad': 'v1', 'squadv2': 'v1'}
SAVE_LOGFILE = './logs/trial1.json'

banchmark_paths = {}
for name, version in BENCHMARKS_INFO.items():
    banchmark_paths[name] = {
        'table': f"../data/{name}/tables/{version}/benchmark.csv",
        'dense_db': f"../data/{name}/dbs/{version}/{DBS_DIR_DENSE_VECTORDB_NAME}",
        'sparse_db':  f"../data/{name}/dbs/{version}/{DBS_DIR_SPARSE_VECTORDB_NAME}"
    }

### 3) Загрузка датасета

In [20]:
# загрузить benchmark-датасет
benchmarks_df = {}
for name, bench_path in banchmark_paths.items():
    benchmarks_df[name] = pd.read_csv(banchmark_paths[name]['table'], sep=';')

# инициализировать класс с метриками
base_dir = '/home/aisummer/ssemenov_workspace/nlp-service'
metrics = ReaderMetrics(base_dir)

# logging
logger = Logger(False)
log = logger.get_logger(__name__)

Loading Meteor...


FileNotFoundError: Couldn't find a module script at /home/aisummer/ssemenov_workspace/nlp-service/src/metrics/meteor/meteor.py.

In [11]:
def get_relevant_ans(df, reader):
    relevant_ans = []
    for i in range(df.shape[0]):
        reader.create_answer(config, configLLM, df['question'][i], df['contexts'][i])
        relevant_ans.append(config.answer)
        if i==5:
            break
    return relevant_ans

In [17]:
benchmarks_score = []
for i, name in enumerate(benchmarks_df.keys()):
    predicted = get_relevant_ans(benchmarks_df[name], reader)
    targets = benchmarks_df[name]['answer'].to_list()

    print(predicted[0])
    print(targets[0])

    rouge_score = metrics.rouge(predicted, targets)
    print("rouge: ", rouge_score)

    bleu_score = metrics.bleu(predicted, targets)
    print("bleu: ", bleu_score)

    meteor = metrics.meteor(predicted, targets)
    print("meteor: ", meteor)

    exact_match = metrics.exact_match(predicted, targets)
    print("exact_match: ", exact_match)

    # посчитать метрики 
    score = {
        'MRR': rouge_score,
        'mAP': bleu_score,
        'Recall': meteor,
        'Precision': exact_match
    }

    benchmarks_score.append(score)

   













Органические останки в протерозойских отложениях представлены известковыми выделениями сине-зелёных водорослей, ходами червей и остатками кишечнополостными.
известковыми выделениями сине-зелёных водорослей


NameError: name 'metrics' is not defined

In [ ]:
 # сохранить результат
if os.path.exists(SAVE_LOGFILE):
    print("Файл существует!")
    raise ValueError

log_data = {'info': BENCHMARKS_INFO, 'params': options.items}

with open(SAVE_LOGFILE, 'w', encoding='utf-8') as fd:
    fd.write(json.dumps(log_data, indent=1))